### This part is hovernet preprocess: After MATLAB part of analysis finished on the MATLAB code, this is for modifying some of the wsi(ndpi)src, roisrc, and ndpisrc so that we're only using the 214 image files that we want to use. The MATLAB all_output_concat was manually made by just copy and pasting contents of the each of the excel output files from the MATLAB workflow. After running hovernet workflow, manually add hovernet result to all_output_concat to match 1090 features.
### These src's are used in the hovernet_json2df.py, just making sure all the src's are correct so that hovernet_json2df.py runs properly!

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import shutil
import pickle
from tqdm import tqdm

xlsrc = r'\\shelter\Kyu\skin_aging\clue_cohort\CLUE_image_list_230207_v2.xlsx'
ndpisrc = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi'
xl = pd.read_excel(xlsrc)
xl_tmp = xl[xl["student score"] > 1]
wsi_list = xl_tmp.filename
# create ndpi path:
wsi_ndpi_list = [os.path.join(ndpisrc, x) for x in wsi_list]
dst_src = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\desired_wsi'
for filename in wsi_ndpi_list:
    shutil.copy(filename, dst_src)
# create dl path:
dlsrc = r'\\shelter\Kyu\skin_aging\clue_cohort\DLmask1um'
wsi_list = [x.replace(".ndpi", ".tif") for x in wsi_list]
wsi_dl_list = [os.path.join(dlsrc, x) for x in wsi_list]
dst_src1 = r'\\shelter\Kyu\skin_aging\clue_cohort\DLmask1um\desired_DLmask'
for filename in wsi_dl_list:
    shutil.copy(filename, dst_src1)
# create roi path:
roisrc = r'\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeledmask_v2_021723'
wsi_list = [x.replace(".tif", ".png") for x in wsi_list]
wsi_roi_list = [os.path.join(roisrc, x) for x in wsi_list]
dst_src2 = r'\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeledmask_v2_021723\desired_roi'
for filename in wsi_roi_list:
    shutil.copy(filename, dst_src2)

### This part is hovernet postprocess: This code belows processes the generated .pkl file from hovernet to the .xlsx file in a form that we desire:

In [2]:
json = pd.read_json(r"\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\json\2022-06-07 13.18.40.json", orient='index')
json = pd.DataFrame(json[0].loc['nuc']).T.drop(columns=['type_prob'])
json = json[json['contour'].map(len) > 5].reset_index(drop=True)
json

,bbox,centroid,contour,type
0,"[[9746, 53047], [9767, 53058]]","[10043.722222222223, 52763.5873015873]","[[10046, 52754], [10045, 52755], [10044, 52755...",None
1,"[[9774, 53087], [9813, 53106]]","[10087.926315789473, 52803.10736842105]","[[10084, 52782], [10083, 52783], [10082, 52783...",None
2,"[[9779, 53057], [9806, 53074]]","[10056.087336244542, 52798.615720524016]","[[10056, 52787], [10055, 52788], [10055, 52789...",None
3,"[[9787, 53031], [9813, 53043]]","[10027.71978021978, 52805.32967032967]","[[10031, 52795], [10030, 52796], [10026, 52796...",None
4,"[[9805, 53057], [9820, 53066]]","[10052.82, 52820.29]","[[10053, 52813], [10052, 52814], [10051, 52814...",None
...,...,...,...,...
78467,"[[84060, 75784], [84074, 75799]]","[83983.33870967742, 75874.3870967742]","[[83980, 75868], [83979, 75869], [83977, 75869...",None
78468,"[[84194, 75960], [84212, 75975]]","[84158.90816326531, 76009.63775510204]","[[84156, 76002], [84155, 76003], [84154, 76003...",None
78469,"[[83998, 81884], [84009, 81893]]","[83936.75862068965, 81955.39655172414]","[[83938, 81950], [83937, 81951], [83937, 81952...",None
78470,"[[86190, 79657], [86203, 79676]]","[85811.52795031056, 80052.32298136647]","[[85812, 80046], [85810, 80048], [85809, 80048...",None


In [136]:
bbox_tmp = json["bbox"].tolist()
for idx in range(len(bbox_tmp)):
    bbox_tmp[idx] = [[int(x/2),int(y/2)] for (x,y) in bbox_tmp[idx]]
json["bbox"] = bbox_tmp

In [137]:
contour_tmp = json["centroid"].tolist()
contour_x1 = [contour_tmp[i][0]/2 for i in range(len(contour_tmp))]
contour_y1 = [contour_tmp[i][1]/2 for i in range(len(contour_tmp))]
new_contour_xy = list(zip(contour_x1,contour_y1))
new_contour_xy = [[new_contour_xy[i][0],new_contour_xy[i][1]] for i in range(len(new_contour_xy))] #,int(new_contour_xy[i][1])
json["centroid"] = new_contour_xy

In [138]:
contour_tmp = json["contour"].tolist()
for idx in range(len(contour_tmp)):
    contour_tmp[idx] = [[int(x/2),int(y/2)] for (x,y) in contour_tmp[idx]]
json["contour"] = contour_tmp

In [139]:
json

,bbox,centroid,contour,type
0,"[[4873, 26523], [4883, 26529]]","[5021.861111111111, 26381.79365079365]","[[5023, 26377], [5022, 26377], [5022, 26377], ...",None
1,"[[4887, 26543], [4906, 26553]]","[5043.9631578947365, 26401.553684210525]","[[5042, 26391], [5041, 26391], [5041, 26391], ...",None
2,"[[4889, 26528], [4903, 26537]]","[5028.043668122271, 26399.307860262008]","[[5028, 26393], [5027, 26394], [5027, 26394], ...",None
3,"[[4893, 26515], [4906, 26521]]","[5013.85989010989, 26402.664835164836]","[[5015, 26397], [5015, 26398], [5013, 26398], ...",None
4,"[[4902, 26528], [4910, 26533]]","[5026.41, 26410.145]","[[5026, 26406], [5026, 26407], [5025, 26407], ...",None
...,...,...,...,...
78467,"[[42030, 37892], [42037, 37899]]","[41991.66935483871, 37937.1935483871]","[[41990, 37934], [41989, 37934], [41988, 37934...",None
78468,"[[42097, 37980], [42106, 37987]]","[42079.454081632655, 38004.81887755102]","[[42078, 38001], [42077, 38001], [42077, 38001...",None
78469,"[[41999, 40942], [42004, 40946]]","[41968.379310344826, 40977.69827586207]","[[41969, 40975], [41968, 40975], [41968, 40976...",None
78470,"[[43095, 39828], [43101, 39838]]","[42905.76397515528, 40026.16149068323]","[[42906, 40023], [42905, 40024], [42904, 40024...",None


In [21]:
import pandas as pd

pkl = pd.read_pickle(r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\2022-06-07 13.18.40.pkl')

In [22]:
pkl

,bbox,centroid,contour,type,inroi,resident_area,c2t_distance,Area,Perimeter,Circularity,...,orientation,AspectRatio,Sol,Extent,EquiDia,imID,dist2nearest,oriA,oriB,local_align
0,"[[25653, 15178], [25669, 15196]]","[26450.82645089286, 14396.988839285714]","[[26450, 14389], [26450, 14389], [26448, 14389...",2,1,95599,"[78.875, 8.0, 2604.0, 469.25, 452.25, 1801.0, ...",216.5,57.213203,0.831142,...,61.602367,1.125654,0.968680,0.707516,16.602902,2022-06-07 13.18.4011337,11.597414,47.785965,54.641518,0.126348
1,"[[25662, 15203], [25673, 15216]]","[26473.506097560974, 14403.181402439024]","[[26474, 14398], [26473, 14399], [26473, 14399...",2,1,95599,"[114.5625, 0.0, 2584.0, 427.5, 412.0, 1764.0, ...",74.0,44.970562,0.459817,...,54.641518,1.931048,0.855491,0.440476,9.706685,2022-06-07 13.18.4011337,10.630146,70.961960,61.602367,0.131239
2,"[[25679, 15150], [25688, 15158]]","[26417.97076023392, 14419.269005847953]","[[26417, 14415], [26416, 14416], [26415, 14416...",2,1,95599,"[50.125, 4.0, 2674.0, 516.5, 513.0, 1868.0, 71...",40.0,28.485281,0.619482,...,21.086744,1.112633,0.879121,0.555556,7.136496,2022-06-07 13.18.4011337,11.800424,59.775425,47.785965,0.461839
3,"[[25680, 15188], [25690, 15197]]","[26456.96712802768, 14421.084775086505]","[[26456, 14416], [26456, 14417], [26455, 14417...",2,1,95599,"[111.125, 0.0, 2630.0, 449.25, 445.25, 1802.0,...",66.5,33.676619,0.736842,...,24.636820,1.116650,0.930070,0.604545,9.201654,2022-06-07 13.18.4011337,3.807887,167.598145,54.641518,0.916066
4,"[[25683, 15182], [25695, 15188]]","[26449.018292682926, 14424.881097560976]","[[26448, 14419], [26447, 14420], [26447, 14422...",2,1,95599,"[103.25, 4.0, 2640.0, 460.0, 457.5, 1815.0, 65...",31.5,29.899495,0.442785,...,167.598145,3.052977,0.828947,0.403846,6.333012,2022-06-07 13.18.4011337,3.807887,24.636820,61.602367,0.877015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11332,"[[37935, 14241], [37945, 14246]]","[37795.515923566876, 14388.210191082802]","[[37795, 14383], [37794, 14385], [37794, 14385...",10,1,304870,"[6400.0, 13576.0, 1420.0, 8832.0, 978.5, 9072....",31.5,24.714777,0.648048,...,1.917912,2.060053,0.887324,0.525000,6.333012,2022-06-07 13.18.4011337,33.105891,111.816551,17.896618,1.353266
11333,"[[37998, 14268], [38002, 14275]]","[37823.27611940299, 14448.059701492537]","[[37822, 14446], [37822, 14446], [37821, 14446...",10,1,304870,"[6392.0, 13624.0, 1307.0, 8808.0, 863.0, 8968....",15.0,16.892922,0.660529,...,111.816551,1.985380,0.882353,0.428571,4.370194,2022-06-07 13.18.4011337,31.148836,17.896618,1.917912,1.353266
11334,"[[38003, 14398], [38013, 14403]]","[37952.200892857145, 14456.25]","[[37951, 14451], [37951, 14452], [37950, 14452...",10,1,304870,"[6588.0, 13848.0, 1145.0, 8992.0, 710.0, 8976....",23.5,24.242641,0.502479,...,166.449829,2.625816,0.870370,0.470000,5.470021,2022-06-07 13.18.4011337,60.170591,17.896618,111.816551,0.761109
11335,"[[37900, 15378], [37904, 15383]]","[37908.6, 15373.86]","[[37909, 15372], [37908, 15373], [37907, 15373...",10,1,304870,"[5932.0, 13768.0, 999.0, 8112.0, 1090.0, 7348....",9.0,13.656854,0.606389,...,48.186665,1.388362,0.857143,0.360000,3.385138,2022-06-07 13.18.4011337,102.753345,84.098717,110.888710,0.388135


In [11]:
bbox_tmp = pkl["bbox"].tolist()
for idx in range(len(bbox_tmp)):
    bbox_tmp[idx] = [[int(x/2),int(y/2)] for (x,y) in bbox_tmp[idx]]
pkl["bbox_20x"] = bbox_tmp
pkl['centroid_20x'] = pkl['centroid'].apply(lambda row: [_/2 for _ in row])
contour_tmp = pkl["contour"].tolist()
for idx in range(len(contour_tmp)):
    contour_tmp[idx] = [[int(x/2),int(y/2)] for (x,y) in contour_tmp[idx]]
pkl["contour_20x"] = contour_tmp

In [1]:
import os
import pickle
from tqdm import tqdm
import pandas as pd
import numpy as np

In [6]:
pkl_src=r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df'
output_src=r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\output_excel'
pkl_list = [x for x in os.listdir(pkl_src) if x.endswith(".pkl")]
pkl_full_list = [os.path.join(pkl_src,x) for x in pkl_list]

for idx, pkl in tqdm(enumerate(range(len(pkl_list))),desc="Pickle File Processed",colour='red',total = len(pkl_list)):
    with open(pkl_full_list[idx], 'rb') as f:
        print("converting pkl to df:")
        df = pickle.load(f)
        bbox_tmp = df["bbox"].tolist()
        for idx1 in range(len(bbox_tmp)):
            bbox_tmp[idx1] = [[int(x/2),int(y/2)] for (x,y) in bbox_tmp[idx1]]
        df["bbox_20x"] = bbox_tmp
        df['centroid_20x'] = df['centroid'].apply(lambda row: [_/2 for _ in row])

        contour_tmp = df["contour"].tolist()
        for idx2 in range(len(contour_tmp)):
            contour_tmp[idx2] = [[int(x/2),int(y/2)] for (x,y) in contour_tmp[idx2]]
        df["contour_20x"] = contour_tmp
    # df dict, key is the type and the value is a DataFrame with the relevant rows
    dfs_by_type = {}
    for t in range(2, 12):
        dfs_by_type[t] = df[df['type'] == t].reset_index(drop=True)

    expanded_dfs = []
    for t, df_t in dfs_by_type.items():
        new_cols = {f"{col}_type{t}": df_t[col] for col in df_t.columns if col != 'type'}
        expanded_dfs.append(pd.DataFrame(new_cols))

    expanded_df = pd.concat(expanded_dfs, axis=1)
    imID,_ = os.path.splitext(pkl_list[idx])
    output_pth = os.path.join(output_src,imID + ".xlsx")
    # save original df
    expanded_df.to_excel(output_pth, index=False)

    print("calculating average")
    # todo: get the distances!

    for col in expanded_df.columns:
        if expanded_df[col].dtype not in [int, float]:
            if col == "inroi_type2":
                continue
            expanded_df.drop(columns=col,inplace=True)
    expanded_df = expanded_df.replace([np.inf, -np.inf], np.nan).dropna()

    grouped = expanded_df.groupby(by=['inroi_type2']).mean()
    print(grouped)
    # create new DataFrame with same column names as expanded_df
    unique_values = expanded_df['inroi_type2'].unique()
    cardinality = len(unique_values)
    avg_df = pd.DataFrame(columns=expanded_df.columns)

    # append a new row for each unique value of inroi_type2
    for value in range(1,cardinality+1):
        if value != value:  # check for NaN value
            continue
        row = grouped.loc[grouped['inroi_type2'] == value].to_dict()
        avg_df = avg_df.append(row, ignore_index=True)

    # append additional rows to avg_df based on the difference between cardinality and the number of unique values
    for value in range(1,cardinality+1):
        if value != value:  # check for NaN value
            continue
        row = expanded_df.loc[expanded_df['inroi_type2'] == value].mean().to_dict()
        avg_df = avg_df.append(row, ignore_index=True)

    # sort avg_df by inroi_type2
    avg_df = avg_df.sort_values('inroi_type2').reset_index(drop=True)

    # drop cols with inroi in them
    inroi_cols = [col for col in df.columns if 'inroi_type' in col]
    avg_df = avg_df.drop(columns=inroi_cols)

    # save average df
    with pd.ExcelWriter(output_pth, engine='openpyxl', mode='a') as writer:
        avg_df.to_excel(writer, sheet_name='averages', index=False)

Pickle File Processed:   0%|          | 0/214 [00:00<?, ?it/s]

converting pkl to df:


Pickle File Processed:   0%|          | 0/214 [00:02<?, ?it/s]

KeyboardInterrupt



In [7]:
import os
import pickle
from tqdm import tqdm
import pandas as pd
import numpy as np
with open(r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\2022-06-07 13.29.45.pkl', 'rb') as f:
    print("converting pkl to df:")
    df = pickle.load(f)
    bbox_tmp = df["bbox"].tolist()
    for idx1 in range(len(bbox_tmp)):
        bbox_tmp[idx1] = [[int(x/2),int(y/2)] for (x,y) in bbox_tmp[idx1]]
    df["bbox_20x"] = bbox_tmp
    df['centroid_20x'] = df['centroid'].apply(lambda row: [_/2 for _ in row])

    contour_tmp = df["contour"].tolist()
    for idx2 in range(len(contour_tmp)):
        contour_tmp[idx2] = [[int(x/2),int(y/2)] for (x,y) in contour_tmp[idx2]]
    df["contour_20x"] = contour_tmp
    # df dict, key is the type and the value is a DataFrame with the relevant rows
    dfs_by_type = {}
    for t in range(2, 12):
        dfs_by_type[t] = df[df['type'] == t].reset_index(drop=True)

    expanded_dfs = []
    for t, df_t in dfs_by_type.items():
        new_cols = {f"{col}_type{t}": df_t[col] for col in df_t.columns if col != 'type'}
        expanded_dfs.append(pd.DataFrame(new_cols))

    expanded_df = pd.concat(expanded_dfs, axis=1)

converting pkl to df:


In [35]:

# todo: get the distances!

for col in expanded_df.columns:
    if expanded_df[col].dtype not in [int, float]:
        if col == "inroi_type2":
            continue
        expanded_df.drop(columns=col,inplace=True)

In [36]:
expanded_df

,inroi_type2,resident_area_type2,Area_type2,Perimeter_type2,Circularity_type2,MA_type2,ma_type2,orientation_type2,AspectRatio_type2,Sol_type2,...,ma_type11,orientation_type11,AspectRatio_type11,Sol_type11,Extent_type11,EquiDia_type11,dist2nearest_type11,oriA_type11,oriB_type11,local_align_type11
0,4,57357,51.5,36.569541,0.483925,14.607458,5.012929,139.064423,2.913956,0.880342,...,5.520935,42.141922,2.081813,0.884615,0.418182,7.653040,8.062258,100.230293,121.238243,0.466269
1,4,57357,303.0,68.404541,0.813734,20.563667,18.966291,67.867828,1.084222,0.968051,...,4.697136,145.923813,1.260763,0.926829,0.527778,4.918491,70.066040,147.179016,115.431717,0.132018
2,4,57357,234.0,63.378203,0.732059,19.714821,14.940122,21.635691,1.319589,0.955102,...,4.583908,64.725357,2.994222,0.834951,0.367521,7.399277,18.973666,47.551579,48.870728,0.177923
3,4,57357,79.0,40.064495,0.618469,11.910135,8.879769,162.085800,1.341266,0.913295,...,5.942382,16.640558,3.248883,0.902564,0.514620,10.585135,26.655206,43.699333,48.536812,0.473651
4,4,57357,149.5,47.963990,0.816621,15.649416,12.774332,173.758316,1.225067,0.961415,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57351,3,141027,71.5,44.242641,0.459022,19.422844,5.474000,2.238300,3.548200,0.910828,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57352,3,141027,54.5,43.899495,0.355375,16.125137,5.686230,106.553688,2.835822,0.741497,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57353,3,141027,39.0,27.721349,0.637743,10.299442,6.197195,87.614426,1.661952,0.876404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57354,3,141027,42.5,27.135563,0.725306,9.056720,6.623593,6.833347,1.367342,0.913978,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
expanded_df = expanded_df.replace([np.inf, -np.inf, np.nan], np.nan)
grouped = expanded_df.groupby(by=['inroi_type2']).nanmean()
grouped

# create new DataFrame with same column names as expanded_df
# unique_values = expanded_df['inroi_type2'].unique()
# cardinality = len(unique_values)
# avg_df = pd.DataFrame(columns=expanded_df.columns)
#
# # append a new row for each unique value of inroi_type2
# for value in range(1,cardinality+1):
#     if value != value:  # check for NaN value
#         continue
#     row = grouped.loc[grouped['inroi_type2'] == value].to_dict()
#     avg_df = avg_df.append(row, ignore_index=True)
#
# # append additional rows to avg_df based on the difference between cardinality and the number of unique values
# for value in range(1,cardinality+1):
#     if value != value:  # check for NaN value
#         continue
#     row = expanded_df.loc[expanded_df['inroi_type2'] == value].mean().to_dict()
#     avg_df = avg_df.append(row, ignore_index=True)
#
# # sort avg_df by inroi_type2
# avg_df = avg_df.sort_values('inroi_type2').reset_index(drop=True)
#
# # drop cols with inroi in them
# inroi_cols = [col for col in df.columns if 'inroi_type' in col]
# avg_df = avg_df.drop(columns=inroi_cols)

AttributeError: 'DataFrameGroupBy' object has no attribute 'nanmean'

### Then calculate CV of all of the values to complete 1090 features:

In [7]:
'c2t_distance_type' in 'c2t_distance_type2'

True

In [10]:
expanded_df

,bbox_type2,centroid_type2,contour_type2,inroi_type2,resident_area_type2,c2t_distance_type2,Area_type2,Perimeter_type2,Circularity_type2,MA_type2,...,Extent_type11,EquiDia_type11,imID_type11,dist2nearest_type11,oriA_type11,oriB_type11,local_align_type11,bbox_20x_type11,centroid_20x_type11,contour_20x_type11
0,"[[47823, 34526], [47834, 34538]]","[47843.716894977166, 34516.93378995434]","[[47839, 34511], [47839, 34511], [47838, 34511...",4,57357,"[71104.0, 70080.0, 67968.0, 67648.0, 65440.0, ...",51.5,36.569541,0.483925,14.607458,...,0.418182,7.653040,2022-06-07 13.29.45162897,8.062258,100.230293,121.238243,0.466269,"[[23124, 19143], [23129, 19148]]","[23241.87684729064, 19030.774630541873]","[[23243, 19028], [23242, 19028], [23242, 19028..."
1,"[[48206, 31595], [48226, 31615]]","[49013.318181818184, 30808.53229665072]","[[49013, 30798], [49013, 30799], [49011, 30799...",4,57357,"[68800.0, 68352.0, 65856.0, 65600.0, 62720.0, ...",303.0,68.404541,0.813734,20.563667,...,0.527778,4.918491,2022-06-07 13.29.45162897,70.066040,147.179016,115.431717,0.132018,"[[32004, 13341], [32007, 13344]]","[31774.84677419355, 13573.771505376344]","[[31774, 13572], [31774, 13572], [31773, 13572..."
2,"[[48234, 31555], [48253, 31572]]","[48971.41365046536, 30835.81385729059]","[[48971, 30826], [48970, 30827], [48969, 30827...",4,57357,"[68864.0, 68416.0, 65920.0, 65728.0, 62816.0, ...",234.0,63.378203,0.732059,19.714821,...,0.367521,7.399277,2022-06-07 13.29.45162897,18.973666,47.551579,48.870728,0.177923,"[[36719, 12238], [36723, 12245]]","[36817.528894472365, 12145.604271356784]","[[36820, 12143], [36820, 12144], [36819, 12144..."
3,"[[48239, 31602], [48251, 31612]]","[49015.17877906977, 30836.77034883721]","[[49015, 30831], [49015, 30831], [49014, 30831...",4,57357,"[68800.0, 68352.0, 65856.0, 65600.0, 62720.0, ...",79.0,40.064495,0.618469,11.910135,...,0.514620,10.585135,2022-06-07 13.29.45162897,26.655206,43.699333,48.536812,0.473651,"[[38639, 13372], [38648, 13376]]","[38461.992875647666, 13555.64378238342]","[[38463, 13551], [38463, 13551], [38462, 13551..."
4,"[[48253, 31598], [48268, 31611]]","[49012.431677018634, 30852.905279503106]","[[49011, 30845], [49010, 30846], [49009, 30846...",4,57357,"[68800.0, 68352.0, 65856.0, 65600.0, 62752.0, ...",149.5,47.963990,0.816621,15.649416,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57351,"[[93268, 27769], [93287, 27775]]","[93307.88819875776, 27742.231366459626]","[[93307, 27732], [93307, 27733], [93306, 27734...",3,141027,"[23440.0, 23024.0, 22720.0, 22784.0, 23728.0, ...",71.5,44.242641,0.459022,19.422844,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57352,"[[93282, 27542], [93289, 27558]]","[93086.25192307692, 27749.084615384614]","[[93080, 27746], [93080, 27746], [93079, 27746...",3,141027,"[23056.0, 22640.0, 22336.0, 22400.0, 23344.0, ...",54.5,43.899495,0.355375,16.125137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57353,"[[93288, 27569], [93295, 27580]]","[93111.32180851063, 27755.172872340427]","[[93111, 27752], [93109, 27754], [93107, 27754...",3,141027,"[23088.0, 22672.0, 22384.0, 22432.0, 23392.0, ...",39.0,27.721349,0.637743,10.299442,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57354,"[[93293, 27556], [93302, 27563]]","[93095.54838709677, 27761.65860215054]","[[93095, 27757], [93092, 27760], [93092, 27763...",3,141027,"[23056.0, 22656.0, 22352.0, 22416.0, 23360.0, ...",42.5,27.135563,0.725306,9.056720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
grouped

NameError: name 'grouped' is not defined

In [5]:
grouped["inroi_type2"]

KeyError: 'inroi_type2'

In [9]:
expanded_df.reset_index()

,index,inroi_type2,resident_area_type2,Area_type2,Perimeter_type2,Circularity_type2,MA_type2,ma_type2,orientation_type2,AspectRatio_type2,...,ma_type11,orientation_type11,AspectRatio_type11,Sol_type11,Extent_type11,EquiDia_type11,dist2nearest_type11,oriA_type11,oriB_type11,local_align_type11
0,0,4,57357,51.5,36.569541,0.483925,14.607458,5.012929,139.064423,2.913956,...,5.520935,42.141922,2.081813,0.884615,0.418182,7.653040,8.062258,100.230293,121.238243,0.466269
1,1,4,57357,303.0,68.404541,0.813734,20.563667,18.966291,67.867828,1.084222,...,4.697136,145.923813,1.260763,0.926829,0.527778,4.918491,70.066040,147.179016,115.431717,0.132018
2,2,4,57357,234.0,63.378203,0.732059,19.714821,14.940122,21.635691,1.319589,...,4.583908,64.725357,2.994222,0.834951,0.367521,7.399277,18.973666,47.551579,48.870728,0.177923
3,3,4,57357,79.0,40.064495,0.618469,11.910135,8.879769,162.085800,1.341266,...,5.942382,16.640558,3.248883,0.902564,0.514620,10.585135,26.655206,43.699333,48.536812,0.473651
